In [ ]:
#pip install konlpy

In [ ]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from konlpy.tag import Twitter
twt = Twitter()
#from hanspell import spell_checker
import re
from konlpy.tag import Okt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

# 데이터 불러오기

In [ ]:
df = pd.read_csv('./result/2020-10-03_20-36-11.csv')
df['date']  = pd.to_datetime(df['date'], format = "%Y-%m-%d").replace("-","")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()
# 평균 3점이며 1댓글당 14정도 도움이 된다고 했지만 편차가 심하기 때문에 조사를 진행

In [ ]:
df

In [ ]:
변화도 = df[['date', 'ratings','helpful']]
변화도 = df.groupby(['date']).agg({'ratings': 'count' , 'helpful':'sum'})
변화도 = 변화도.reset_index()
변화도.set_index('date',inplace = True)
변화도

In [ ]:
변화도.resample('M').sum().plot()

In [ ]:
변화도.info()

In [ ]:
df.sort_values(by='helpful',ascending=False).head(20)

In [ ]:
변화도

In [ ]:
변화도['helpful'].sort_values(ascending= False).head(10)

In [ ]:
변화도.resample('M').sum().describe()

# 댓글수는 월 22개정도이지만 도움이 된다는 수는 지속적으로 상승하고있다

# helpful 은 공감이 되는 글인걸 확인 할 수 있다.

### 필요한 컬럼 및 결측치 제거

In [ ]:
df = df[['date','comment','ratings','helpful']]
df = df.dropna()
df.isnull().sum()

In [ ]:
def text_cleaning(text):
    hangul = re.compile('[|ㄱ-ㅎ|ㅏ-ㅣ]+') # 한글의 정규표현식을 나타냅니다.
    result = hangul.sub('', text)
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글의 정규표현식을 나타냅니다.
    result = hangul.sub('', text)
    return result

In [ ]:
df['ko_text'] = df['comment'].apply(lambda x: text_cleaning(x))
df.head()

In [ ]:
# 한국어 약식 불용어사전 예시 파일입니다. 출처 - (https://www.ranks.nl/stopwords/korean)
korean_stopwords_path = "./korean_stopwords.txt"
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

def get_nouns(x):
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(x)
    
    # 한글자 키워드를 제거합니다.
    nouns = [noun for noun in nouns if len(noun) > 1]
    
    # 불용어를 제거합니다.
    nouns = [noun for noun in nouns if noun not in stopwords]
    
    return nouns

In [ ]:
df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x))
print(df.shape)
df.head()

In [ ]:
df.info()

In [ ]:
df2014 = df.query("date >= '2014-01-01' and date <='2014-12-31'")
df2015= df.query("date >= '2015-01-01' and date <='2015-12-31'")
df2016= df.query("date >= '2016-01-01' and date <='2016-12-31'")
df2017= df.query("date >= '2017-01-01' and date <='2017-12-31'")
df2018= df.query("date >= '2018-01-01' and date <='2018-12-31'")
df2019= df.query("date >= '2019-01-01' and date <='2019-12-31'")
df2020= df.query("date >= '2020-01-01' and date <='2020-12-31'")
df2014.reset_index(drop= True, inplace= True)
df2015.reset_index(drop= True, inplace= True)
df2016.reset_index(drop= True, inplace= True)
df2017.reset_index(drop= True, inplace= True)
df2018.reset_index(drop= True, inplace= True)
df2019.reset_index(drop= True, inplace= True)
df2020.reset_index(drop= True, inplace= True)


In [ ]:
#all_words_df2014~2020로 변수 저장
c = 2014
for i in [df2014,df2015,df2016,df2017,df2018,df2019,df2020]:
    
    all_words = []
    satisfaction= []
    for j in range(len(i)):
        words = i['nouns'][j]
        words_arr = []
        for n in words:
            words_arr.append(n)
            satisfaction.append(i['ratings'].iloc[j])
        all_words.extend(words_arr)
       
    globals()['all_words_df{}'.format(c) ]= pd.DataFrame({"words":all_words, "satisfaction":satisfaction, "count":len(all_words)*[1]})
    
    c = c+ 1 


In [ ]:
all_words_df2014

In [ ]:
words_satisfaction = all_words_df2014.groupby('words').mean()['satisfaction']
words_count = all_words_df2014.groupby('words').sum()['count']
words_df = pd.concat([words_satisfaction, words_count], axis=1)
words_df.sort_values('count',ascending=False)

In [ ]:
words_satisfaction = all_words_df2015.groupby('words').mean()['satisfaction']
words_count = all_words_df2015.groupby('words').sum()['count']
words_df = pd.concat([words_satisfaction, words_count], axis=1)
words_df.sort_values('count',ascending=False)

In [ ]:
words_satisfaction = all_words_df2016.groupby('words').mean()['satisfaction']
words_count = all_words_df2016.groupby('words').sum()['count']
words_df = pd.concat([words_satisfaction, words_count], axis=1)
words_df.sort_values('count',ascending=False)

In [ ]:
words_satisfaction = all_words_df2017.groupby('words').mean()['satisfaction']
words_count = all_words_df2017.groupby('words').sum()['count']
words_df = pd.concat([words_satisfaction, words_count], axis=1)
words_df.sort_values('count',ascending=False)

In [ ]:
words_satisfaction = all_words_df2018.groupby('words').mean()['satisfaction']
words_count = all_words_df2018.groupby('words').sum()['count']
words_df = pd.concat([words_satisfaction, words_count], axis=1)
words_df.sort_values('count',ascending=False)

In [ ]:
words_satisfaction = all_words_df2019.groupby('words').mean()['satisfaction']
words_count = all_words_df2019.groupby('words').sum()['count']
words_df = pd.concat([words_satisfaction, words_count], axis=1)
words_df.sort_values('count',ascending=False)

In [ ]:
words_satisfaction = all_words_df2020.groupby('words').mean()['satisfaction']
words_count = all_words_df2020.groupby('words').sum()['count']
words_df = pd.concat([words_satisfaction, words_count], axis=1)
words_df.sort_values('count',ascending=False)

In [ ]:
stopwords = "./korean_stopwords.txt"
nouns_tagger =Okt()
def text_cleaning(text):
  hangul = re.compile('[^ ㄱ-ㅣ가-힣]')
  result = hangul.sub('', text)
  tagger = Okt()
  nouns = nouns_tagger.nouns(result)
  nouns = [x for x in nouns if len(x) >1]
  nouns =[x for x in nouns if x not in stopwords]
  return nouns

vect = CountVectorizer(tokenizer= lambda x : text_cleaning(x))
bow_vect = vect.fit_transform(df['comment'].tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

In [ ]:
vect14 = CountVectorizer(tokenizer= lambda x : text_cleaning(x))
bow_vect2014 = vect14.fit_transform(df2014['comment'].tolist())
word_list2014 = vect14.get_feature_names()
count_list2014 = bow_vect2014.toarray().sum(axis=0)

In [ ]:
vect15 = CountVectorizer(tokenizer= lambda x : text_cleaning(x))
bow_vect2015 = vect15.fit_transform(df2015['comment'].tolist())
word_list2015 = vect15.get_feature_names()
count_list2015 = bow_vect2015.toarray().sum(axis=0)

In [ ]:
vect16 = CountVectorizer(tokenizer= lambda x : text_cleaning(x))

bow_vect2016 = vect16.fit_transform(df2016['comment'].tolist())
word_list2016 = vect16.get_feature_names()
count_list2016 = bow_vect2016.toarray().sum(axis=0)

In [ ]:
vect17 = CountVectorizer(tokenizer= lambda x : text_cleaning(x))

bow_vect2017 = vect17.fit_transform(df2017['comment'].tolist())
word_list2017 = vect17.get_feature_names()
count_list2017 = bow_vect2017.toarray().sum(axis=0)

In [ ]:
vect18 = CountVectorizer(tokenizer= lambda x : text_cleaning(x))

bow_vect2018 = vect18.fit_transform(df2018['comment'].tolist())
word_list2018 = vect18.get_feature_names()
count_list2018 = bow_vect2018.toarray().sum(axis=0)

In [ ]:
vect19 = CountVectorizer(tokenizer= lambda x : text_cleaning(x))

bow_vect2019 = vect19.fit_transform(df2019['comment'].tolist())
word_list2019 = vect19.get_feature_names()
count_list2019 = bow_vect2019.toarray().sum(axis=0)

In [ ]:
vect20 = CountVectorizer(tokenizer= lambda x : text_cleaning(x))

bow_vect2020 = vect20.fit_transform(df2020['comment'].tolist())
word_list2020 = vect20.get_feature_names()
count_list2020 = bow_vect.toarray().sum(axis=0)

In [ ]:
print('2014',bow_vect2014.shape)
print('2015',bow_vect2015.shape)
print('2016',bow_vect2016.shape)
print('2017',bow_vect2017.shape)
print('2018',bow_vect2018.shape)
print('2019',bow_vect2019.shape)
print('2020',bow_vect2020.shape)

In [ ]:
for i in range(0,7):
    wl = [word_list2014,word_list2015,word_list2016,word_list2017,word_list2018,word_list2019,word_list2020]
    cl = [count_list2014,count_list2015,count_list2016,count_list2017,count_list2018,count_list2019,count_list2020]
    word_count_dict = dict(zip(wl[i], cl[i]))
    print(i,str(word_count_dict)[:100])

In [ ]:
tfidf_vectorizer = TfidfTransformer()
s=2014
for i in [bow_vect2014,bow_vect2015,bow_vect2016,bow_vect2017,bow_vect2018,bow_vect2019,bow_vect2020]:
    
    globals()[f'tf_idf_vect{s}'] = tfidf_vectorizer.fit_transform(i)
    s=s+ 1

In [ ]:
print(2014,tf_idf_vect2014[0])
print(2015,tf_idf_vect2015[0])
print(2016,tf_idf_vect2016[0])
print(2017,tf_idf_vect2017[0])
print(2018,tf_idf_vect2018[0])
print(2019,tf_idf_vect2019[0])
print(2020,tf_idf_vect2020[0])


In [ ]:
invert_index_vectorizer = {v:k for k,v in vect.vocabulary_.items()}
print(str(invert_index_vectorizer)[:100])

In [ ]:
invert_index_vectorizer14 = {v:k for k,v in vect14.vocabulary_.items()}
invert_index_vectorizer15 = {v:k for k,v in vect15.vocabulary_.items()}
invert_index_vectorizer16 = {v:k for k,v in vect16.vocabulary_.items()}
invert_index_vectorizer17 = {v:k for k,v in vect17.vocabulary_.items()}
invert_index_vectorizer18 = {v:k for k,v in vect18.vocabulary_.items()}
invert_index_vectorizer19 = {v:k for k,v in vect19.vocabulary_.items()}
invert_index_vectorizer20 = {v:k for k,v in vect20.vocabulary_.items()}

print('14',str(invert_index_vectorizer14)[:100])
print('15',str(invert_index_vectorizer15)[:100])
print('16',str(invert_index_vectorizer16)[:100])
print('17',str(invert_index_vectorizer17)[:100])
print('18',str(invert_index_vectorizer18)[:100])
print('19',str(invert_index_vectorizer19)[:100])
print('20',str(invert_index_vectorizer20)[:100])


In [ ]:
ax1 = plt.subplot(331)
graph1 =df2014.ratings.hist()
ax1.set_title('2014')

ax2 = plt.subplot(332)
graph2 =df2015.ratings.hist()
ax2.set_title('2015')

ax3 = plt.subplot(333)
graph3 =df2016.ratings.hist()
ax3.set_title('2016')

ax4 = plt.subplot(334)
graph4 =df2017.ratings.hist()
ax4.set_title('2017')

ax5 = plt.subplot(335)
graph5 =df2018.ratings.hist()
ax5.set_title('2018')

ax6 = plt.subplot(336)
graph6 =df2019.ratings.hist()
ax6.set_title('2019')

ax7 = plt.subplot(337)
graph7 =df2020.ratings.hist()
ax7.set_title('2020')

plt.tight_layout()

In [ ]:
def rating_to_label(rating) :
  if rating >3:
    return 1
  else:
    return 0
df2014['y'] = df2014['ratings'].apply(lambda x : rating_to_label(x))
df2015['y'] = df2015['ratings'].apply(lambda x : rating_to_label(x))
df2016['y'] = df2016['ratings'].apply(lambda x : rating_to_label(x))
df2017['y'] = df2017['ratings'].apply(lambda x : rating_to_label(x))
df2018['y'] = df2018['ratings'].apply(lambda x : rating_to_label(x))
df2019['y'] = df2019['ratings'].apply(lambda x : rating_to_label(x))
df2020['y'] = df2020['ratings'].apply(lambda x : rating_to_label(x)) 
df['y'] = df['ratings'].apply(lambda x : rating_to_label(x))

In [ ]:
print('전체',df.y.value_counts())
print('2014\n',df2014.y.value_counts())
print('2015\n',df2015.y.value_counts())
print('2016\n',df2016.y.value_counts())
print('2017\n',df2017.y.value_counts())
print('2018\n',df2018.y.value_counts())
print('2019\n',df2019.y.value_counts())
print('2020\n',df2020.y.value_counts())

In [ ]:
y= df2020['y']
x_train,x_test, y_train, y_test = train_test_split(tf_idf_vect2020, y, test_size = 0.3)
print(x_test.shape)
lr = LogisticRegression(random_state=0)
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
confmat = confusion_matrix(y_test,y_pred)


In [ ]:
print('accuracy_score',accuracy_score(y_test,y_pred))
print('precision_score', precision_score(y_test,y_pred))
print('recall_score', recall_score(y_test,y_pred))
print('f1_score',f1_score(y_test,y_pred))
print('confmat', confmat)

In [ ]:
# positive_random_idx = df[df['y']==1].sample(892, random_state=33).index.tolist()
# negative_random_idx = df[df['y']==0].sample(892,random_state=33).index.tolist()
# random_idx = positive_random_idx+negative_random_idx

# x=tf_idf_vect[random_idx]
# y=df['y'][random_idx]
# x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
lr= LogisticRegression(random_state =0)
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)

In [ ]:
plt.rcParams['figure.figsize'] = [10,8]
plt.bar(range(len(lr.coef_[0])), lr.coef_[0])

In [ ]:
print(sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse=True)[:5])
print(sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse=True)[-5:])


In [ ]:
coef_pos_index = sorted(((value, index ) for index, value in enumerate(lr.coef_[0])), reverse=True)
coef_neg_index = sorted(((value,index) for index, value in enumerate(lr.coef_[0])), reverse= False)

In [ ]:
for coef in coef_pos_index[:20]:
  print(invert_index_vectorizer[coef[1]], coef[0])

In [ ]:
for coef in coef_neg_index[:20]:
  print(invert_index_vectorizer[coef[1]], coef[0])